<a href="https://colab.research.google.com/github/DataScientistMSBA/APIs/blob/main/NOAA_API_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Source](https://github.com/paulokuong/noaa)

**PreLoads**

In [7]:
def PreLoads():
  !pip install noaa-sdk

  global clear_output, datetime, time, pd, NOAA, n
  import datetime; import time 
  from IPython.display import clear_output
  import pandas as pd
  from noaa_sdk import NOAA
  n = NOAA()
PreLoads()

time.sleep(1)
clear_output(wait=True)
print('PreLoads processed')

PreLoads processed


**List of columns available**

In [8]:
res = n.get_forecasts('11365', 'US', type='forecastGridData')
col_list = []
for i in res:
    col_list.append(i)
str(col_list)

"['@id', '@type', 'updateTime', 'validTimes', 'elevation', 'forecastOffice', 'gridId', 'gridX', 'gridY', 'temperature', 'dewpoint', 'maxTemperature', 'minTemperature', 'relativeHumidity', 'apparentTemperature', 'heatIndex', 'windChill', 'skyCover', 'windDirection', 'windSpeed', 'windGust', 'weather', 'hazards', 'probabilityOfPrecipitation', 'quantitativePrecipitation', 'iceAccumulation', 'snowfallAmount', 'snowLevel', 'ceilingHeight', 'visibility', 'transportWindSpeed', 'transportWindDirection', 'mixingHeight', 'hainesIndex', 'lightningActivityLevel', 'twentyFootWindSpeed', 'twentyFootWindDirection', 'waveHeight', 'wavePeriod', 'waveDirection', 'primarySwellHeight', 'primarySwellDirection', 'secondarySwellHeight', 'secondarySwellDirection', 'wavePeriod2', 'windWaveHeight', 'dispersionIndex', 'pressure', 'probabilityOfTropicalStormWinds', 'probabilityOfHurricaneWinds', 'potentialOf15mphWinds', 'potentialOf25mphWinds', 'potentialOf35mphWinds', 'potentialOf45mphWinds', 'potentialOf20mphWi

**Get historical data from global historical climatology network data**

In [9]:
observations = n.get_observations('80202','US',start='2017-01-01',end='2018-02-02')
for observation in observations:
    print(observation)
    # break

**Get recent data**

In [10]:
Zip = '80202'
res = n.get_observations(Zip,'US',start='2021-01-01',end='2022-06-13')
df = []
for i in res:
  time = i['timestamp'][11:16]
  date = datetime.datetime.strptime(i['timestamp'][0:10], '%Y-%m-%d').strftime("%m/%d/%Y")
  weather = i['textDescription']
  temp = i['temperature']['value']
  dewpoint = i['dewpoint']['value']
  windDirection = i['windDirection']['value']
  windSpeed = i['windSpeed']['value']
  windGust = i['windGust']['value']
  barometricPressure = i['barometricPressure']['value'] #/133.322
  seaLevelPressure = i['seaLevelPressure']['value']
  visibility = i['visibility']['value']
  minTemperatureLast24Hours = i['minTemperatureLast24Hours']['value']
  maxTemperatureLast24Hours = i['maxTemperatureLast24Hours']['value']
  precipitationLastHour = i['precipitationLastHour']['value'] # / .3048 % 1 * 12
  precipitationLast3Hours = i['precipitationLast3Hours']['value'] # / .3048 % 1 * 12
  precipitationLast6Hours = i['precipitationLast6Hours']['value'] # / .3048 % 1 * 12
  relativeHumidity = i['relativeHumidity']['value']
  windChill = i['windChill']['value']
  heatIndex = i['heatIndex']['value']

  df.append([date,time,weather,temp,dewpoint,windDirection,windSpeed,windGust,barometricPressure,seaLevelPressure,visibility,minTemperatureLast24Hours,maxTemperatureLast24Hours,precipitationLastHour,precipitationLast3Hours,precipitationLast6Hours,relativeHumidity,windChill,heatIndex])
df = pd.DataFrame(df).rename(columns={0:'Date', 1:'Time', 2:'Weather', 3:'Celsius', 4:'Dew Point', 5:'Wind Direction', 6:'Wind Speed', 7:'Wind Gust', 8:'Barometric Pressure temp', 9:'Sea Level Pressure temp', 10:'Visibility', 11:'24HrMinTemp', 12:'24HrMaxTemp', 13:'1Hr Precipitation', 14:'3Hr Precipitation', 15:'6Hr Precipitation', 16:'Relative Humidity', 17:'Wind Chill', 18:'Heat Index'}).sort_values(by=['Date','Time']).reset_index()
df = df.assign(Fahrenheit = lambda x: (9/5)*x['Celsius']+32).rename(columns={'Fahrenheit':'Temperature'})[['Date','Time','Weather','Temperature','Dew Point','Wind Direction','Wind Speed','Wind Gust','Barometric Pressure temp','Sea Level Pressure temp','Visibility','24HrMinTemp','24HrMaxTemp','1Hr Precipitation','3Hr Precipitation','6Hr Precipitation','Relative Humidity','Wind Chill','Heat Index']]

# df = df.assign(temp = lambda x: (9/5)*x['Wind Chill']+32).rename(columns={'temp':'Wind Chill'})[['Date','Time','Weather','Temperature','Dew Point','Wind Direction','Wind Speed','Wind Gust','Barometric Pressure','Sea Level Pressure','Visibility','24HrMinTemp','24HrMaxTemp','1Hr Precipitation','3Hr Precipitation','6Hr Precipitation','Relative Humidity','Wind Chill','Heat Index']]
df = df.assign(temp = lambda x: x['Barometric Pressure temp']/.3048 % 1 * 12).rename(columns={'temp':'Barometric Pressure'})[['Date','Time','Weather','Temperature','Dew Point','Wind Direction','Wind Speed','Wind Gust','Barometric Pressure','Sea Level Pressure temp','Visibility','24HrMinTemp','24HrMaxTemp','1Hr Precipitation','3Hr Precipitation','6Hr Precipitation','Relative Humidity','Wind Chill','Heat Index']]
df = df.assign(temp = lambda x: x['Sea Level Pressure temp']/.3048 % 1 * 12).rename(columns={'temp':'Sea Level Pressure'})[['Date','Time','Weather','Temperature','Dew Point','Wind Direction','Wind Speed','Wind Gust','Barometric Pressure','Sea Level Pressure','Visibility','24HrMinTemp','24HrMaxTemp','1Hr Precipitation','3Hr Precipitation','6Hr Precipitation','Relative Humidity','Wind Chill','Heat Index']]
df

,Date,Time,Weather,Temperature,Dew Point,Wind Direction,Wind Speed,Wind Gust,Barometric Pressure,Sea Level Pressure,Visibility,24HrMinTemp,24HrMaxTemp,1Hr Precipitation,3Hr Precipitation,6Hr Precipitation,Relative Humidity,Wind Chill,Heat Index
0,06/06/2022,22:49,Light Thunderstorms and Light Rain,66.2,11.0,270.0,14.76,NaN,1.196850,NaN,16090.0,None,None,None,None,None,59.762816,None,NaN
1,06/06/2022,23:45,Light Thunderstorms and Light Rain,62.6,9.0,310.0,22.32,NaN,1.196850,NaN,16090.0,None,None,None,None,None,59.282135,None,NaN
2,06/07/2022,00:45,Mostly Cloudy,66.2,8.0,130.0,18.36,NaN,0.188976,NaN,16090.0,None,None,None,None,None,48.856133,None,NaN
3,06/07/2022,01:15,Mostly Cloudy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16090.0,None,None,None,None,None,NaN,None,NaN
4,06/07/2022,01:50,Mostly Cloudy,69.8,4.0,310.0,18.36,NaN,10.393701,NaN,16090.0,None,None,None,None,None,32.745738,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,06/13/2022,17:47,Smoke,91.4,-3.0,110.0,12.96,NaN,8.724409,NaN,12870.0,None,None,None,None,None,9.755808,None,30.202972
248,06/13/2022,18:47,Smoke,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11270.0,None,None,None,None,None,NaN,None,NaN
249,06/13/2022,19:47,Smoke,95.0,-3.0,150.0,12.96,NaN,9.511811,NaN,12870.0,None,None,None,None,None,8.725404,None,31.714441
250,06/13/2022,20:54,Smoke,95.0,-2.0,130.0,14.76,25.92,5.700787,NaN,12870.0,None,None,None,None,None,9.395899,None,31.818846
